# Data Loading

In [3]:
from database_wrapper import DatabaseWrapper
from sreality_scraper.sreality.spiders.sreality_spider import SrealityUrlBuilder
import sys
import pandas as pd
import numpy as np
import unidecode

# get the data from the database
db = DatabaseWrapper("listings.db")
df = db.get_df()
db.close_conn()
if df is None:
    print("No data in the database")
    sys.exit(1)

# Data Cleaning

In [4]:
# set id column as index
df.set_index("id", inplace=True)

In [5]:
# drop public_transport column, it's not useful for now
df.drop(columns=["public_transport"], inplace=True)
# drop security_deposit column
df.drop(columns=["security_deposit"], inplace=True)

df.drop(columns=["pets"], inplace=True)
df.drop(columns=["service_fees"], inplace=True)

## Dispositions

In [6]:
# print unique disposition values
df.disposition.unique()

array([47, 6, 2, 7, 4, 8, 16, 5, 9, 11, 3, '2+kk', '4+kk', '3+kk', '2+1',
       '1+1', 'Ostatní', 10, '1+kk', '4+1', '3+1', 12, '5+1', '5+kk',
       'Garsoniéra', None], dtype=object)

In [7]:
# map integers (sreality disposition ids) to strings
df.disposition = df.disposition.apply(
    lambda x: SrealityUrlBuilder.map_category_sub_cb(x) if isinstance(x, int) else x
)
df.disposition.unique()

array(['pokoj', '3+kk', '1+kk', '3+1', '2+kk', '4+kk', 'atypicky', '2+1',
       '4+1', '5+1', '1+1', 'Ostatní', '5+kk', '6-a-vice', 'Garsoniéra',
       None], dtype=object)

In [8]:
# unify disposition values
df.disposition = (
    df.disposition.replace("Garsoniéra", "1+kk")
    .replace("Ostatní", "other")
    .replace("atypicky", "other")
    .replace("pokoj", "other")
    .replace("6+kk", "6-a-více")
    .replace("6+1", "6-a-více")
    .replace("7+kk", "6-a-více")
    .replace("7+1", "6-a-více")
)
df.disposition.unique()

array(['other', '3+kk', '1+kk', '3+1', '2+kk', '4+kk', '2+1', '4+1',
       '5+1', '1+1', '5+kk', '6-a-vice', None], dtype=object)

## Area

In [9]:
df.area.sort_values()

id
3031377228      8.0
222887          9.0
3597452620     10.0
2319983948     10.0
2386367820     10.0
              ...  
2769237324    308.0
1763648844    386.0
3815060812    421.0
2434610508    435.0
828640          NaN
Name: area, Length: 5245, dtype: float64

## Available from

In [10]:
df.available_from.unique()

array(['Ihned', '01.04.2024', '01.06.2024', '01.07.2024', '01.05.2024',
       '22.03.2024', '11. 3. 2024', '10. 3. 2024', '1. 2. 2024',
       '15.04.2024', '9. 8. 2023', '21. 3. 2024', '1. 3. 2024', None,
       '1. 4. 2024', '17. 3. 2024', '20. 2. 2024', '20. 3. 2024',
       '7. 6. 2024', '12. 5. 2024', '5. 3. 2024', '1. 5. 2024',
       '19. 2. 2024', '16. 3. 2024', '15. 3. 2024', '30.04.2024',
       '25. 3. 2024', '8. 4. 2024', '1. 6. 2024', '4. 3. 2024',
       '2. 5. 2024', '11. 5. 2024', '25. 4. 2024', '15. 5. 2024',
       '6. 3. 2024', '31. 3. 2024', '2. 3. 2024', '14. 2. 2024',
       '20. 4. 2024', '15. 4. 2024', '1. 9. 2023', '12. 3. 2024',
       '12. 1. 2024', '14. 3. 2024', '23. 2. 2024', '8. 3. 2024',
       '17. 5. 2024', '30. 4. 2024', '6. 4. 2024', '30. 3. 2024',
       '28. 3. 2024', '29. 3. 2024', '18. 3. 2024', '10. 5. 2024',
       '08.04.2024', '14. 4. 2024', '26. 1. 2023', '1. 2. 2023',
       '13. 3. 2024', '10.05.2024', '3. 4. 2024', '13. 5. 2024',
       

In [11]:
# trim whitespaces
df.available_from = df.available_from.str.replace(" ", "")

In [12]:
# replace "Ihned" with last updated date
ihned_rows = df["available_from"] == "Ihned"
df.loc[df["available_from"] == "Ihned", "available_from"] = pd.to_datetime(
    df.loc[df["available_from"] == "Ihned", "updated"],
    format="%Y-%m-%d %H:%M:%S.%f",
    errors="coerce",
)

In [13]:
# try parsing the date for all non empty fields
invalid_indices = pd.to_datetime(df.available_from, errors="coerce").isnull()

In [14]:
invalid_indices = invalid_indices | ihned_rows
df.loc[~invalid_indices, "available_from"].unique()

array(['01.04.2024', '01.06.2024', '01.07.2024', '01.05.2024',
       '22.03.2024', '11.3.2024', '10.3.2024', '1.2.2024', '15.04.2024',
       '9.8.2023', '21.3.2024', '1.3.2024', '1.4.2024', '17.3.2024',
       '20.2.2024', '20.3.2024', '7.6.2024', '12.5.2024', '5.3.2024',
       '1.5.2024', '19.2.2024', '16.3.2024', '15.3.2024', '30.04.2024',
       '25.3.2024', '8.4.2024', '1.6.2024', '4.3.2024', '2.5.2024',
       '11.5.2024', '25.4.2024', '15.5.2024', '6.3.2024', '31.3.2024',
       '2.3.2024', '14.2.2024', '20.4.2024', '15.4.2024', '1.9.2023',
       '12.3.2024', '12.1.2024', '14.3.2024', '23.2.2024', '8.3.2024',
       '17.5.2024', '30.4.2024', '6.4.2024', '30.3.2024', '28.3.2024',
       '29.3.2024', '18.3.2024', '10.5.2024', '08.04.2024', '14.4.2024',
       '26.1.2023', '1.2.2023', '13.3.2024', '10.05.2024', '3.4.2024',
       '13.5.2024', '7.3.2024', '29.2.2024', '16.1.2024', '23.11.2023',
       '27.2.2024', '25.2.2024', '14.5.2024', '1.10.2023', '9.4.2024',
       '1.9.202

In [15]:
df.loc[~invalid_indices, "available_from"] = pd.to_datetime(
    df.loc[~invalid_indices, "available_from"],
    format="%d.%m.%Y",
    errors="coerce",
)

In [16]:
df.loc[~invalid_indices, "available_from"]

id
225629516     2024-04-01 00:00:00
4060792140    2024-06-01 00:00:00
2784609612    2024-04-01 00:00:00
2122564940    2024-04-01 00:00:00
4263016268    2024-07-01 00:00:00
                     ...         
328078668     2024-05-20 00:00:00
3364754764    2024-04-08 00:00:00
419239244     2024-04-01 00:00:00
1810699596    2024-03-21 00:00:00
1660753228    2024-06-01 00:00:00
Name: available_from, Length: 2093, dtype: object

In [17]:
# get all NaT values from available_from
df.loc[df.available_from.isnull()]

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,last_seen,loggie,ownership,parking,rent,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
489409,"Přemyslovská, Praha - Vinohrady",90.0,None,Balkón 20 m²,Sklep 8 m²,2024-03-20 09:40:25.001979,bydlete v unikátním (plně vybaveném) bytě situ...,2+kk,Výtah,6,...,2024-03-20 12:18:36.783776,None,Osobní,Parkování,34 000 Kč,Velmi dobrý,Terasa 20 m²,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
508735,"Makedonská, Praha - Střížkov",82.0,None,Balkón 10 m²,None,2024-03-20 09:40:25.001979,nabízíme k pronájmu byt 3kk o rozloze 72m2 plu...,3+kk,Výtah,3,...,2024-03-20 12:18:36.783776,None,Osobní,None,28 000 Kč,None,None,Ostatní,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
528745,"Hellichova, Praha - Malá Strana",52.0,None,Balkón 4 m²,Sklep 8 m²,2024-03-20 09:40:25.001979,nájemní bytová jednotka 2+kk na malé straně s ...,2+kk,None,2,...,2024-03-20 12:18:36.783776,None,Osobní,None,28 000 Kč,None,None,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
543869,"U Milosrdných, Praha - Staré Město",71.0,None,None,None,2024-03-20 09:40:25.001979,"luxusně kompletně zařízený a vybavený 2+kk, po...",2+kk,Výtah,5,...,2024-03-20 12:18:36.783776,None,Osobní,None,32 000 Kč,None,None,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
79624,"Spálená, Praha - Nové Město",107.0,None,None,Sklep 6 m²,2024-03-20 09:40:25.001979,"praha 1 byt v secesním domě, na okraji pěší z...",4+kk,None,1,...,2024-03-20 12:18:36.783776,Lodžie 7 m²,Osobní,None,31 500 Kč,None,None,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
541329,"V Podhájí, Praha - Podolí",130.0,None,Balkón 3 m²,None,2024-03-20 09:40:25.001979,nabízíme k pronájmu krásný mezonetový byt 4+1 ...,4+1,None,2,...,2024-03-20 12:18:36.783776,None,Osobní,Parkování,34 990 Kč,Velmi dobrý,Terasa 15 m²,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
510642,"Hošťálkova, Praha - Břevnov",55.0,None,None,None,2024-03-20 09:40:25.001979,zrekonstruovaný byt je v prvním patře rohové ř...,2+1,None,1,...,2024-03-20 12:18:36.783776,None,Osobní,None,22 000 Kč,Po rekonstrukci,None,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
448166,"Lyčkovo náměstí, Praha - Karlín",89.0,None,"Balkón 3,1 m²",None,2024-03-20 09:40:25.001979,pronajmu byt 3+kk ve 3.patře na lyčkově náměst...,3+kk,None,3,...,2024-03-20 12:18:36.783776,None,Osobní,None,22 000 Kč,Velmi dobrý,None,Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...
552335,"Korunovační, Praha - Bubeneč",70.0,None,Balkón 3 m²,None,2024-03-20 09:40:25.001979,"nabízíme k pronájmu prostorný světlý byt 2+1, ...",2+1,None,1,...,2024-03-20 12:18:36.783776,None,Osobní,None,24 000 Kč,None,"Terasa 0,1 m²",Cihla,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...


## Balcony

In [18]:
df.balcony.unique()

array([0, 1, None, 'Balkón 18 m²', 'Balkón 7 m²', 'Balkón 20 m²',
       'Balkón 23 m²', 'Balkón 1,5 m²', 'Balkón 2 m²', 'Balkón 5 m²',
       'Balkón 4 m²', 'Balkón 8 m²', 'Balkón 8,5 m²', 'Balkón 3,5 m²',
       'Balkón', 'Balkón 14 m²', 'Balkón 10 m²', 'Balkón 6 m²',
       'Balkón 7,2 m²', 'Balkón 3 m²', 'Balkón 6,4 m²', 'Balkón 2,5 m²',
       'Balkón 12 m²', 'Balkón 10,5 m²', 'Balkón 12,1 m²',
       'Balkón 3,2 m²', 'Balkón 19 m²', 'Balkón 1 m²', 'Balkón 9 m²',
       'Balkón 5,8 m²', 'Balkón 11 m²', 'Balkón 15 m²', 'Balkón 13,1 m²',
       'Balkón 31 m²', 'Balkón 13 m²', 'Balkón 5,6 m²', 'Balkón 6,8 m²',
       'Balkón 5,1 m²', 'Balkón 3,6 m²', 'Balkón 25 m²', 'Balkón 3,1 m²',
       'Balkón 3,4 m²', 'Balkón 4,7 m²', 'Balkón 7,3 m²', 'Balkón 6,1 m²',
       'Balkón 4,2 m²', 'Balkón 2,1 m²', 'Balkón 5,5 m²', 'Balkón 6,6 m²',
       'Balkón 2,25 m²', 'Balkón 5,79 m²', 'Balkón 4,02 m²',
       'Balkón 3,14 m²', 'Balkón 4,86 m²', 'Balkón 3,45 m²',
       'Balkón 4,1 m²', 'Balkón 8,

In [19]:
# if there's a mention of balk in the field, then balcony is most likely present
df.balcony = df.balcony.apply(
    lambda x: 1 if isinstance(x, str) and "balk" in x.lower() else x
)

In [20]:
df.balcony.unique()

array([ 0.,  1., nan])

## Cellar

In [21]:
df.cellar.unique()

array([0, 1, None, 'Sklep', 'Sklep 3 m²', 'Sklep 4 m²', 'Sklep 8 m²',
       'Sklep 6 m²', 'Sklep 2,9 m²', 'Sklep 1,2 m²', 'Sklep 2,5 m²',
       'Sklep 10 m²', 'Sklep 2 m²', 'Sklep 9 m²', 'Sklep 2,2 m²',
       'Sklep 2,15 m²', 'Sklep 5 m²', 'Sklep 1 m²', 'Sklep 15 m²',
       'Sklep 20 m²', 'Sklep 4,7 m²', 'Sklep 4,5 m²', 'Sklep 4,8 m²',
       'Sklep 3,5 m²', 'Sklep 1,5 m²', 'Sklep 3,6 m²', 'Sklep 1,9 m²',
       'Sklep 7 m²', 'Sklep 4,19 m²', 'Sklep 2,1 m²', 'Sklep 3,2 m²',
       'Sklep 3,8 m²', 'Sklep 3,1 m²', 'Sklep 2,4 m²', 'Sklep 3,4 m²',
       'Sklep 42 m²', 'Sklep 2,27 m²', 'Sklep 5,1 m²', 'Sklep 2,3 m²',
       'Sklep 1,45 m²', 'Sklep 2,64 m²', 'Sklep 1,4 m²', 'Sklep 16 m²',
       'Sklep 1,6 m²', 'Sklep 1,1 m²', 'Sklep 4,1 m²', 'Sklep 60 m²',
       'Sklep 2,8 m²', 'Sklep 4,4 m²', 'Sklep 28 m²', 'Sklep 1,3 m²',
       'Sklep 1,7 m²', 'Sklep 80 m²'], dtype=object)

In [22]:
df.cellar = df.cellar.apply(
    lambda x: 1 if isinstance(x, str) and "sklep" in x.lower() else x
)

In [23]:
df.cellar.unique()

array([ 0.,  1., nan])

## Balcony

In [24]:
df.balcony.unique()

array([ 0.,  1., nan])

## Elevator

In [25]:
df.elevator.unique()

array([1, 2, 0, None, 'Výtah'], dtype=object)

In [26]:
df.elevator = df.elevator.apply(
    lambda x: 1 if isinstance(x, str) and "výtah" in x.lower() else x
).apply(lambda x: 0 if x == 2 else x)

In [27]:
df.elevator.unique()

array([ 1.,  0., nan])

## Floor

In [28]:
df.floor.unique()

array(['3. podlaží z celkem 8', '3. podlaží', '-1. podlaží', '4. podlaží',
       '2. podlaží', '4. podlaží z celkem 4', '2. podlaží z celkem 3',
       '2. podlaží z celkem 5', '3. podlaží z celkem 3',
       '2. podlaží z celkem 4', '2. podlaží z celkem 6',
       '6. podlaží z celkem 5', '1. podlaží z celkem 3',
       '3. podlaží z celkem 6', '1. podlaží',
       '6. podlaží z celkem 9 včetně 1 podzemního',
       '4. podlaží včetně 1 podzemního',
       '5. podlaží z celkem 5 včetně 2 podzemních',
       '4. podlaží z celkem 5', '6. podlaží z celkem 7', '6. podlaží',
       '4. podlaží z celkem 6', '6. podlaží z celkem 6',
       '3. podlaží z celkem 5', '5. podlaží z celkem 6',
       '1. podlaží z celkem 5', '5. podlaží z celkem 5',
       '1. podlaží z celkem 1', '5. podlaží z celkem 7', '7. podlaží',
       '1. podlaží z celkem 4',
       '1. podlaží z celkem 3 včetně 1 podzemního',
       '6. podlaží z celkem 8',
       '5. podlaží z celkem 8 včetně 1 podzemního', '5. podlaží

In [29]:
df.loc[:, "floor"] = df.floor.replace(". podlaží.*", "", regex=True).replace(
    " z celkem.*", "", regex=True
)
df.floor.unique()

array(['3', '-1', '4', '2', '6', '1', '5', '7', '10', '8', 'přízemí',
       '15', '9', '21', None, '19', '11', '12', '20', '13', '-2', '17',
       '14', '136', '18'], dtype=object)

In [30]:
df.floor = df.floor.apply(lambda x: 0 if x == "přízemí" else x).apply(
    lambda x: np.nan if x == None else int(x)
)
df.floor.unique()

array([  3.,  -1.,   4.,   2.,   6.,   1.,   5.,   7.,  10.,   8.,   0.,
        15.,   9.,  21.,  nan,  19.,  11.,  12.,  20.,  13.,  -2.,  17.,
        14., 136.,  18.])

## Garden

In [31]:
df.garden.unique()

array([None, 'Předzahrádka 34 m²', 450, 5, 189, 63, 320, 60,
       'Předzahrádka 300 m²', 1000, 52, 1850, 'Předzahrádka 72 m²',
       'Předzahrádka 40,1 m²', 'Předzahrádka 15 m²',
       'Předzahrádka 100 m²', 122, 'Předzahrádka 200 m²',
       'Předzahrádka 42 m²', 'Předzahrádka 55 m²', 'Předzahrádka 98 m²',
       'Předzahrádka 500 m²', 'Předzahrádka 700 m²',
       'Předzahrádka 110 m²', 'Předzahrádka 20 m²', 'Předzahrádka 150 m²',
       'Předzahrádka 16 m²', 'Předzahrádka 81 m²', 'Předzahrádka 10 m²',
       37, 50, 'Předzahrádka 40 m²', 'Předzahrádka 80 m²',
       'Předzahrádka 1 m²', 1500, 'Předzahrádka 116 m²', 14, 30,
       'Předzahrádka 26 m²', 'Předzahrádka 18 m²', 'Předzahrádka 23 m²',
       48, 'Předzahrádka 6 m²', 'Předzahrádka 27 m²',
       'Předzahrádka 50 m²', 400, 72, 'Předzahrádka 30 m²',
       'Předzahrádka 8 m²', 27, 'Předzahrádka 400 m²',
       'Předzahrádka 7 m²', 54, 'Předzahrádka 4 m²', 401,
       'Předzahrádka 115 m²', 2000, 600, 16, 316, 8, 26, 499, 

In [32]:
# TODO: remove unidecode and move it to Listings class
df.garden = df.garden.apply(
    lambda x: (
        unidecode.unidecode(x)
        .replace("Predzahradka ", "")
        .replace(" m2", "")
        .replace(" ", "")
        .replace(",", ".")
        if isinstance(x, str)
        else x
    )
).astype(float)
df.loc[df.garden.notnull(), "garden"]

id
782819         34.0
1046095180    450.0
3521590604      5.0
2609263948    189.0
3345298764     63.0
              ...  
3764831564    100.0
2488063308    100.0
730398        350.0
3010573644     14.0
2820478284     32.0
Name: garden, Length: 165, dtype: float64

## Furnished

In [33]:
df.furnished.unique()

array([1, 3, 2, 0, 'Částečně', 'Vybaveno', None, 'Nevybaveno'],
      dtype=object)

In [34]:
df.furnished = df.furnished.apply(
    lambda x: (
        x.replace("Nevybaveno", "2").replace("Částečně", "3").replace("Vybaveno", "1")
        if isinstance(x, str)
        else x
    )
).apply(lambda x: np.nan if x == None else int(x))

In [35]:
df.furnished.unique()

array([ 1.,  3.,  2.,  0., nan])

## Garage

In [36]:
df.garage.unique()

array([0, 1, None, 'Garáž'], dtype=object)

In [37]:
df.garage = df.garage.apply(lambda x: 1 if isinstance(x, str) and "Garáž" in x else x)
df.garage.unique()

array([ 0.,  1., nan])

## Loggie

In [38]:
df.loggie.unique()

array([0, 1, None, 'Lodžie', 'Lodžie 13 m²', 'Lodžie 4,1 m²',
       'Lodžie 6 m²', 'Lodžie 5 m²', 'Lodžie 12 m²', 'Lodžie 2,5 m²',
       'Lodžie 4 m²', 'Lodžie 7 m²', 'Lodžie 8 m²', 'Lodžie 1,9 m²',
       'Lodžie 12,5 m²', 'Lodžie 3,6 m²', 'Lodžie 3 m²', 'Lodžie 20 m²',
       'Lodžie 5,18 m²', 'Lodžie 4,4 m²', 'Lodžie 9 m²', 'Lodžie 2 m²',
       'Lodžie 3,2 m²', 'Lodžie 24,1 m²', 'Lodžie 6,1 m²',
       'Lodžie 4,7 m²', 'Lodžie 5,6 m²', 'Lodžie 3,1 m²', 'Lodžie 1,5 m²',
       'Lodžie 11,8 m²', 'Lodžie 7,5 m²', 'Lodžie 5,79 m²',
       'Lodžie 2,1 m²', 'Lodžie 3,95 m²', 'Lodžie 4,2 m²',
       'Lodžie 6,6 m²', 'Lodžie 3,5 m²', 'Lodžie 6,5 m²', 'Lodžie 10 m²'],
      dtype=object)

In [39]:
df.loggie = df.loggie.apply(
    lambda x: 1 if isinstance(x, str) and "Lodžie" in x else x
).astype(float)
df.loggie.unique()

array([ 0.,  1., nan])

## Parking

In [40]:
df.parking.unique()

array([0, 1, None, 'Parkování'], dtype=object)

In [41]:
df.parking = df.parking.apply(
    lambda x: 1 if isinstance(x, str) and "Parkování" in x else x
).astype(float)
df.parking.unique()

array([ 0.,  1., nan])

## Rent

In [42]:
df.rent.unique()

array([10694, 28000, 13000, 65689, 31000, 16000, 40000, 64800, 32000,
       36300, 34900, 29000, 34990, 38000, 15000, 60000, 53057, 63163,
       39000, 58000, 18000, 26000, 17000, 24500, 48000, 58110, 14300,
       35000, 30000, 34000, 62400, 84000, 57000, 25000, 12000, 80000,
       95000, 19800, None, 100000, 28490, 120000, 60270, 88000, 16500,
       59600, 36000, 55583, 49000, 99000, 57700, 33000, 19000, 55000,
       22000, 25500, 19999, 20000, 27000, 11500, 23700, 21900, 50000,
       45000, 24000, 90000, 23500, 21300, '35 500 Kč', 22900, '42 000 Kč',
       '41 700 Kč', 20400, '41 000 Kč', 8000, '40 817 Kč', '30 000 Kč',
       25900, '34 000 Kč', '29 000 Kč', '28 799 Kč', '28 765 Kč',
       '26 990 Kč', '26 900 Kč', '26 800 Kč', '26 609 Kč', '26 500 Kč',
       '26 490 Kč', 49900, '26 000 Kč', 19900, 39800, '27 000 Kč', 18500,
       32999, 14000, 21500, 79000, 12500, 44000, 65000, 27500, 24200,
       109800, 16900, 42500, '28 000 Kč', 33500, 7500, 13200, 64190,
       2488

In [43]:
df.loc[:, "rent"] = df.rent.apply(
    lambda x: (
        int(x.replace(" ", "").replace("Kč", "").replace("€", ""))
        if isinstance(x, str)
        else x
    )
).astype(float)

df.rent.sort_values().unique()

array([4000.0, 5000.0, 5100.0, 5200.0, 5300.0, 5500.0, 6000.0, 6250.0,
       6500.0, 6700.0, 6716.0, 7000.0, 7500.0, 7600.0, 7667.0, 7900.0,
       7999.0, 8000.0, 8200.0, 8400.0, 8500.0, 8700.0, 8870.0, 8900.0,
       8990.0, 9000.0, 9310.0, 9358.0, 9393.0, 9500.0, 9600.0, 9700.0,
       9800.0, 9900.0, 9963.0, 9990.0, 10000.0, 10158.0, 10400.0, 10490.0,
       10500.0, 10694.0, 10900.0, 10990.0, 11000.0, 11410.0, 11490.0,
       11500.0, 11750.0, 11800.0, 11900.0, 11990.0, 12000.0, 12200.0,
       12300.0, 12499.0, 12500.0, 12600.0, 12700.0, 12745.0, 12800.0,
       12870.0, 12872.0, 12880.0, 12900.0, 12990.0, 13000.0, 13190.0,
       13200.0, 13300.0, 13400.0, 13404.0, 13490.0, 13500.0, 13575.0,
       13600.0, 13700.0, 13750.0, 13800.0, 13855.0, 13890.0, 13900.0,
       13901.0, 13990.0, 14000.0, 14205.0, 14250.0, 14290.0, 14300.0,
       14400.0, 14490.0, 14500.0, 14600.0, 14700.0, 14750.0, 14764.0,
       14800.0, 14900.0, 14990.0, 14999.0, 15000.0, 15200.0, 15300.0,
       1538

## Status

In [44]:
# 'status'

# unique values for status
df.status.unique()

array(['Velmi dobrý', 'Novostavba', 'Po rekonstrukci', 'Dobrý',
       'Před rekonstrukcí', None, 'V rekonstrukci', 'Projekt',
       'Ve výstavbě'], dtype=object)

In [45]:
## Ownership

In [46]:
df.ownership.unique()

array([1, 2, None, 'Osobní', 3, 'Družstevní', 'Ostatní', 'Obecní'],
      dtype=object)

In [47]:
df.ownership = df.ownership.apply(
    lambda x: (
        str(x).replace("1", "Osobní").replace("2", "Družstevní").replace("3", "Ostatní").replace("Obecní", "Ostatní")
        if isinstance(x, int)
        else x
    )
).apply(lambda x: np.nan if x == None else x)

In [48]:
df.ownership.unique()

array(['Osobní', 'Družstevní', nan, 'Ostatní', 'Obecní'], dtype=object)

## Terrace

In [49]:
df.terrace.unique()

array([0, 1, 'Terasa 30 m²', None, 'Terasa 8 m²', 'Terasa 20 m²',
       'Terasa 40 m²', 'Terasa 5,1 m²', 'Terasa 9 m²', 'Terasa 11 m²',
       'Terasa 16 m²', 'Terasa', 'Terasa 6 m²', 'Terasa 18,3 m²',
       'Terasa 7,5 m²', 'Terasa 60 m²', 'Terasa 10 m²', 'Terasa 3 m²',
       'Terasa 53 m²', 'Terasa 12 m²', 'Terasa 14 m²', 'Terasa 70 m²',
       'Terasa 17 m²', 'Terasa 15 m²', 'Terasa 5 m²', 'Terasa 27 m²',
       'Terasa 7 m²', 'Terasa 4 m²', 'Terasa 81 m²', 'Terasa 13 m²',
       'Terasa 45 m²', 'Terasa 5,2 m²', 'Terasa 200 m²', 'Terasa 18 m²',
       'Terasa 82 m²', 'Terasa 0,1 m²', 'Terasa 24 m²', 'Terasa 39 m²',
       'Terasa 80 m²', 'Terasa 112 m²', 'Terasa 21 m²', 'Terasa 25 m²',
       'Terasa 1\xa0000 m²', 'Terasa 65 m²', 'Terasa 23 m²',
       'Terasa 10,5 m²', 'Terasa 7,2 m²', 'Terasa 50 m²', 'Terasa 300 m²',
       'Terasa 2 m²', 'Terasa 3,1 m²', 'Terasa 8,6 m²'], dtype=object)

In [50]:
# 'terrace',
df.loc[:, "terrace"] = df["terrace"].apply(
    lambda x: 1 if isinstance(x, str) and "Terasa" in x else x
)
df.loc[:, "terrace"].unique()

array([0.0, 1.0, nan], dtype=object)

## Type

In [51]:
# 'type'
df.type.unique()
df.loc[:, "type"] = df["type"].apply(
    lambda x: (
        x.replace("Cihla", "cihlova")
        .replace("Panel", "panelova")
        .replace("Smíšená", "ostatni")
        .replace("Skeletová", "ostatni")
        .replace("Nízkoenergetická", "ostatni")
        .replace("Montovaná", "ostatni")
        .replace("Dřevostavba", "ostatni")
        .replace("Kamenná", "ostatni")
        .replace("Ostatní", "ostatni")
        if isinstance(x, str)
        else x
    )
)
df.loc[:, "type"] = df["type"].apply(
    lambda x: SrealityUrlBuilder.map_building_type(x) if isinstance(x, int) else x
)
df.type.unique()

array(['ostatni', 'cihlova', 'panelova', None], dtype=object)

## Floor

In [52]:
df.floor = df.floor.apply(lambda x: int(x) if isinstance(x, str) else x)
df.floor

id
858048604      3.0
3269453132     3.0
4037891148    -1.0
866014284      4.0
829482316      2.0
              ... 
3907851596    18.0
1810699596     6.0
3355252044     2.0
3875345740     2.0
1660753228     3.0
Name: floor, Length: 5245, dtype: float64

## POI Distance

In [53]:
from app import get_point
POI = "NTK Praha"
poi_point = get_point(POI)

In [54]:

from geopy.distance import distance

points_of_interest = [poi_point]

# https://stackoverflow.com/questions/37885798/how-to-calculate-the-midpoint-of-several-geolocations-in-python
x = 0.0
y = 0.0
z = 0.0

for point in points_of_interest:
    
    
    latitude = np.radians(point.latitude)
    longitude = np.radians(point.longitude)

    x += np.cos(latitude) * np.cos(longitude)
    y += np.cos(latitude) * np.sin(longitude)
    z += np.sin(latitude)

total = len(points_of_interest)

x = x / total
y = y / total
z = z / total

central_longitude = np.degrees(np.arctan2(y, x))
central_square_root = np.sqrt(x * x + y * y)
central_latitude = np.degrees(np.arctan2(z, central_square_root))

print(f"{central_latitude}, {central_longitude}")


for i, row in df.iterrows():
    df.loc[i, "poi_distance"] = distance((central_latitude, central_longitude), (row.gps_lat, row.gps_lon)).m
df.poi_distance

50.10391984999999, 14.390643481864164


id
858048604     3363.900532
3269453132    9127.526142
4037891148    8723.113095
866014284     4293.132364
829482316     3140.007238
                 ...     
3907851596    6846.649294
1810699596    8484.763950
3355252044    4078.580807
3875345740    3231.902166
1660753228    2990.164543
Name: poi_distance, Length: 5245, dtype: float64

In [55]:
nominal = ['address', 'description', 'disposition', 'ownership', 'status', 'type', 'url']
ordinal = ['floor', 'furnished', 'balcony', 'cellar', 'elevator', 'garage', 'loggie', 'parking', 'terrace']
interval = ['available_from', 'created', 'last_seen', 'updated']
ratio = ['area', 'rent', 'poi_distance', 'garden'] # 'gps_lat', 'gps_lon' not included


In [56]:
for col in ordinal + ratio:
    print(f"{df[col].sort_values().value_counts(bins=5 if col not in ordinal else None)}\n")

floor
 2.0      1115
 3.0      1069
 4.0       863
 1.0       698
 5.0       633
 6.0       463
 7.0       148
 8.0        72
 9.0        39
 0.0        26
 11.0       23
-1.0        20
 10.0       16
 12.0       12
 15.0        7
-2.0         4
 19.0        4
 13.0        3
 17.0        2
 14.0        2
 21.0        2
 18.0        1
 20.0        1
 136.0       1
Name: count, dtype: int64

furnished
1.0    2392
3.0    1282
2.0     990
0.0     559
Name: count, dtype: int64

balcony
0.0    2973
1.0    1498
Name: count, dtype: int64

cellar
0.0    2975
1.0    1567
Name: count, dtype: int64

elevator
1.0    3467
0.0    1340
Name: count, dtype: int64

garage
0.0    3280
1.0    1074
Name: count, dtype: int64

loggie
0.0    3842
1.0     461
Name: count, dtype: int64

parking
0.0    3313
1.0    1158
Name: count, dtype: int64

terrace
0.0    3436
1.0     886
Name: count, dtype: int64

(7.572, 93.4]     4087
(93.4, 178.8]      961
(178.8, 264.2]     160
(264.2, 349.6]      33
(349.6, 435.0]     

In [57]:
df.fillna(value=np.nan, inplace=True)

/tmp/ipykernel_7220/1829536916.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(value=np.nan, inplace=True)


In [58]:
# save df to xlsx openpyxl==3.1.2 required
df.to_excel("listings.xlsx", na_rep="NaN")
df

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,loggie,ownership,parking,rent,status,terrace,type,updated,url,poi_distance
id,,,,,,,,,,,,,,,,,,,,,
858048604,"Na zátorách, Praha 7 - Holešovice",26.0,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,na realitách uvádíme celkovou měsíční all-in c...,other,1.0,3.0,...,0.0,Osobní,0.0,10694.0,Velmi dobrý,0.0,ostatni,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/po...,3363.900532
3269453132,"Kolbenova, Praha",80.0,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,k pronájmu nabízíme byt o dispozici 3+kk na pr...,3+kk,1.0,3.0,...,0.0,Osobní,0.0,28000.0,Novostavba,1.0,ostatni,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/3+...,9127.526142
4037891148,"Černokostelecká, Praha 10 - Strašnice",33.0,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,"nabízíme k dlouhodobému pronájmu byt 1+kk, kte...",1+kk,0.0,-1.0,...,0.0,Osobní,0.0,13000.0,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/1+...,8723.113095
866014284,"Italská, Praha 2 - Vinohrady",92.0,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,"pronájem kompletně zařízeného bytu 3+kk, 92 m2...",3+kk,1.0,4.0,...,0.0,Osobní,0.0,65689.0,Velmi dobrý,0.0,ostatni,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/3+...,4293.132364
829482316,"U Nesypky, Praha 5 - Smíchov",125.0,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,"zařízený, zrekonstruovaný byt 3+1 s terasou v ...",3+1,0.0,2.0,...,0.0,Osobní,0.0,31000.0,Velmi dobrý,1.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/3+...,3140.007238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3907851596,"Sluneční náměstí, Praha 5 - Stodůlky",110.0,2024-03-20 12:18:36.783776,0.0,1.0,2024-03-20 12:18:36.783776,ihned k pronájmu je připravený prostorný byt s...,3+1,1.0,18.0,...,0.0,Družstevní,1.0,45000.0,Velmi dobrý,0.0,ostatni,2024-03-20 12:18:36.783776,https://www.sreality.cz/detail/pronajem/byt/3+...,6846.649294
1810699596,"náměstí Olgy Scheinpflugové, Praha",42.0,2024-03-21 00:00:00.000000,0.0,0.0,2024-03-20 12:18:36.783776,"nabízíme k pronájmu luxusní, nový a plně zaříz...",1+kk,1.0,6.0,...,0.0,Osobní,0.0,20900.0,Velmi dobrý,1.0,cihlova,2024-03-20 12:18:36.783776,https://www.sreality.cz/detail/pronajem/byt/1+...,8484.763950
3355252044,"Pod Kavalírkou, Praha 5 - Košíře",55.0,2024-03-20 12:18:36.783776,1.0,1.0,2024-03-20 12:18:36.783776,dovolujeme si vám nabídnout k dlouhodobému pro...,2+1,0.0,2.0,...,1.0,Osobní,0.0,17990.0,Dobrý,0.0,cihlova,2024-03-20 12:18:36.783776,https://www.sreality.cz/detail/pronajem/byt/2+...,4078.580807


In [59]:
# mapping = {
#     "1+1": 1,
#     "1+kk": 2,
#     "2+1": 3,
#     "2+kk": 4,
#     "3+1": 5,
#     "3+kk": 6,
#     "4+1": 7,
#     "4+kk": 8,
#     "5+kk": 9,
#     "5+1": 10,
#     "6-a-více": 11,
#     "other": 12,
# }
# df.disposition = df.disposition.map(mapping)


# status_mapping = {
#     "Projekt": 1,
#     "Ve výstavbě": 2,
#     "Novostavba": 3,
#     "Velmi dobrý": 4,
#     "Dobrý": 5,
#     "V rekonstrukci": 6,
#     "Po rekonstrukci": 7,
#     "Před rekonstrukcí": 8,
# }
# df.status = df.status.map(status_mapping)

# type_mapping = {
#     "cihlova": 1,
#     "panelova": 2,
#     "ostatni": 3,
# }
# df.type = df.type.map(type_mapping)

# Normalize columns

In [60]:
for col in ordinal + ratio:
    print(col)
    max_val = df[col].max()
    min_val = df[col].min()
    denominator = max_val - min_val
    if denominator == 0:
        denominator = 1e-10  # Add a small epsilon value to avoid division by zero
    df[col] = (df[col] - min_val) / denominator

    print(df[col].value_counts(bins=10, sort=False))


floor
(-0.002, 0.1]    5189
(0.1, 0.2]         34
(0.2, 0.3]          0
(0.3, 0.4]          0
(0.4, 0.5]          0
(0.5, 0.6]          0
(0.6, 0.7]          0
(0.7, 0.8]          0
(0.8, 0.9]          0
(0.9, 1.0]          1
Name: count, dtype: int64
furnished
(-0.002, 0.1]     559
(0.1, 0.2]          0
(0.2, 0.3]          0
(0.3, 0.4]       2392
(0.4, 0.5]          0
(0.5, 0.6]          0
(0.6, 0.7]        990
(0.7, 0.8]          0
(0.8, 0.9]          0
(0.9, 1.0]       1282
Name: count, dtype: int64
balcony
(-0.002, 0.1]    2973
(0.1, 0.2]          0
(0.2, 0.3]          0
(0.3, 0.4]          0
(0.4, 0.5]          0
(0.5, 0.6]          0
(0.6, 0.7]          0
(0.7, 0.8]          0
(0.8, 0.9]          0
(0.9, 1.0]       1498
Name: count, dtype: int64
cellar
(-0.002, 0.1]    2975
(0.1, 0.2]          0
(0.2, 0.3]          0
(0.3, 0.4]          0
(0.4, 0.5]          0
(0.5, 0.6]          0
(0.6, 0.7]          0
(0.7, 0.8]          0
(0.8, 0.9]          0
(0.9, 1.0]       1567
Name: count

In [61]:
df.to_excel("normalized_listings.xlsx", na_rep="NaN")

# Calculate score

In [62]:
# sum values of all ordinal and ratio columns
df["sum"] = df[ordinal + ratio].sum(axis=1)
# normalize the sum
df["sum"] = (df["sum"] - df["sum"].min()) / (df["sum"].max() - df["sum"].min())

In [63]:
print(ordinal + ratio)
# sort df by sum
pd.set_option("display.max_columns", None)
df.sort_values(by="poi_distance", ascending=False)

['floor', 'furnished', 'balcony', 'cellar', 'elevator', 'garage', 'loggie', 'parking', 'terrace', 'area', 'rent', 'poi_distance', 'garden']


,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,furnished,garage,garden,gps_lat,gps_lon,last_seen,loggie,ownership,parking,rent,status,terrace,type,updated,url,poi_distance,sum
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1207248204,"Únorová, Praha 10 - Nedvězí u Říčan",0.201405,2024-03-22 00:00:00.000000,0.0,0.0,2024-03-20 09:40:25.001979,k dlouhodobému podnájmu nabízíme krásný byt po...,3+1,0.0,0.021739,0.666667,0.0,NaN,50.016275,14.659007,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.097744,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/3+...,1.000000,0.227138
3065783628,"Únorová, Praha 10 - Nedvězí u Říčan",0.016393,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,nabízíme vám k pronájmu byt 1+kk po kompletní ...,1+kk,0.0,0.021739,1.000000,0.0,NaN,50.017538,14.655101,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.022519,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/1+...,0.985381,0.234560
493384,"Holekova, Praha - Klánovice",0.067916,2024-04-01 00:00:00.000000,1.0,NaN,2024-03-20 09:40:25.001979,"pronajmu na dva měsíce světlý, velký, plně vyb...",1+kk,NaN,0.021739,1.000000,NaN,0.004,50.100822,14.687103,2024-03-20 12:18:36.783776,NaN,Osobní,1.0,0.045113,Novostavba,1.0,cihlova,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...,0.984344,0.625136
2656908620,"Žíšovská, Praha 9 - Újezd nad Lesy",0.142857,2024-05-17 00:00:00.000000,1.0,1.0,2024-03-20 09:40:25.001979,vzhledem k velmi vysokému zájmu o námi nabízen...,3+kk,1.0,0.057971,1.000000,0.0,NaN,50.071144,14.678484,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.056353,Po rekonstrukci,0.0,panelova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/3+...,0.970671,0.638431
711364,"Žíšovská, Praha - Újezd nad Lesy",0.142857,2024-05-17 00:00:00.000000,1.0,1.0,2024-03-20 09:40:25.001979,vzhledem k velmi vysokému zájmu o námi nabízen...,3+kk,1.0,0.057971,1.000000,NaN,NaN,50.071144,14.678484,2024-03-20 12:18:36.783776,NaN,Osobní,NaN,0.056353,Po rekonstrukci,NaN,panelova,2024-03-20 09:40:25.001979,https://www.bezrealitky.cz/nemovitosti-byty-do...,0.970671,0.638431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054114124,"Studentská, Praha 6 - Dejvice",0.035129,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,nabízíme vám k pronájmu byt po celkové rekonst...,1+kk,1.0,0.043478,0.666667,0.0,NaN,50.102846,14.390194,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.056391,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/1+...,0.000000,0.203543
1532826956,"Studentská, Praha 6 - Dejvice",0.035129,2024-03-20 09:40:25.001979,0.0,0.0,2024-03-20 09:40:25.001979,nabízíme k pronájmu kompletně zrekonstruovaný ...,1+kk,1.0,0.043478,0.000000,0.0,NaN,50.102846,14.390194,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.056391,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/1+...,0.000000,0.118922
1602819404,"Studentská, Praha 6 - Dejvice",0.037471,2024-03-20 09:40:25.001979,0.0,1.0,2024-03-20 09:40:25.001979,"krásný, kompletně nově zrekonstruovaný, nezaří...",1+kk,1.0,0.036232,0.666667,0.0,NaN,50.102846,14.390194,2024-03-20 12:18:36.783776,0.0,Osobní,0.0,0.056391,Po rekonstrukci,0.0,cihlova,2024-03-20 09:40:25.001979,https://www.sreality.cz/detail/pronajem/byt/1+...,0.000000,0.329851
